In [1]:
! python -m pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 38.8 MB/s eta 0:00:00


In [2]:
from gurobipy import Model, GRB, quicksum

# Insiemi
I = ["L1", "L2"]              # linee di produzione
J = ["Auto", "Moto", "Bici"]  # prodotti

# Parametri

c = {
    ("L1", "Auto"): 6,
    ("L1", "Moto"): 4,
    ("L1", "Bici"): 2,

    ("L2", "Auto"): 5,
    ("L2", "Moto"): 3,
    ("L2", "Bici"): 1,
}

a = {
    ("L1", "Auto"): 2,
    ("L1", "Moto"): 4,
    ("L1", "Bici"): 6,

    ("L2", "Auto"): 3,
    ("L2", "Moto"): 5,
    ("L2", "Bici"): 7,
}

b = {
    "Auto": 100,
    "Moto": 200,
    "Bici": 300,
}

f = {
    "L1": 10,
    "L2": 15,
}

d = {
    "L1": 800,
    "L2": 900,
}

M = max(d.values())

# Modello
model = Model("Produzione_Linee")

## Variabili
x = model.addVars(I, J, vtype=GRB.INTEGER, lb=0, name="x")  # ore dedicate
y = model.addVars(I, vtype=GRB.BINARY, name="y")  # attivazione linea

## Vincoli

### (1)
for j in J:
    model.addConstr(
        quicksum(a[(i, j)] * x[(i, j)] for i in I) >= b[j],
        name=f"domanda_{j}"
    )

### (2)
for i in I:
    model.addConstr(
        quicksum(x[(i, j)] for j in J) <= d[i],
        name=f"capacita_{i}"
    )

### (3)
for i in I:
    model.addConstr(
        quicksum(x[(i, j)] for j in J) <= M * y[i],
        name=f"attivazione_{i}"
    )

## Funzione obiettivo
model.setObjective(
    quicksum(c[(i, j)] * x[(i, j)] for i in I for j in J) +
    quicksum(f[i] * y[i] for i in I),
    GRB.MINIMIZE
)

# Risoluzione
model.optimize()

# Stampa soluzione
if model.status == GRB.OPTIMAL:
    print(f"Costo totale ottimo: {model.objVal:.2f}")
    print("\nAttivazione linee:")
    for i in I:
        print(f"  {i}: y = {int(y[i].X)}")

    print("\nOre di produzione:")
    for i in I:
        for j in J:
            if x[(i, j)].X > 0.5:
                print(f"  Linea {i}, Prodotto {j}: x = {x[(i, j)].X:.0f} ore")


Restricted license - for non-production use only - expires 2027-11-29
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 8 columns and 20 nonzeros (Min)
Model fingerprint: 0xb40b3c2c
Model has 8 linear objective coefficients
Variable types: 0 continuous, 8 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 9e+02]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 14 nonzeros
Variable types: 0 continuous, 8 integer (2 binary)
Found heuristic solution: objective 610.0000000
Found heuristic solution: objective 348.0000000

Root relaxation: objective 3.444200e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |